In [34]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Показывать все столбцы
df_train = pd.read_csv('../src/train1.csv')
X_train = df_train.iloc[:,1:-1]
y_train = df_train.iloc[:,-1]
X_test = pd.read_csv('../src/test1.csv').iloc[:,1:-1]
y_test = pd.read_csv('../src/submission.csv').drop('Id',axis=1)

In [35]:
categorical_columns = X_train.select_dtypes(include=['object', 'category'])
num_columns = X_train.drop([*categorical_columns],axis=1)
Q1 = num_columns.quantile(0.25)
Q3 = num_columns.quantile(0.75)
IQR = Q3 - Q1
num_columns = num_columns[~((num_columns < (Q1 - 1.5 * IQR)) | (num_columns > (Q3 + 1.5 * IQR)))]
num_columns = num_columns.fillna(num_columns.mean()) 

threshold = 0.8
categorical_columns = categorical_columns.dropna(axis=1, thresh=len(categorical_columns) - threshold) # если много пропусков в категориальном столбце

for x in categorical_columns:
    categorical_columns[x] = categorical_columns[x].fillna(categorical_columns[x].mode()[0])

In [36]:
# !pip install category_encoders
from category_encoders import TargetEncoder
enc = TargetEncoder()
categorical_columns = enc.fit_transform(categorical_columns,y_train) 

In [37]:
# убираем признаки с маленькой вариацией (дисперсией)
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

# проверяем и категориальные и числовые.
scaler1 = StandardScaler()
scaler2 = StandardScaler()
num_scaled = pd.DataFrame(scaler1.fit_transform(num_columns),columns = num_columns.columns)
cat_sc = pd.DataFrame(scaler2.fit_transform(categorical_columns),columns = categorical_columns.columns)
sel1 = VarianceThreshold(threshold = 0.05)
sel2 = VarianceThreshold(threshold = 0.05)
print(cat_sc.shape,num_scaled.shape)

sel1.fit(num_scaled)
sel2.fit(cat_sc)
cols1 = num_scaled.columns[sel1.get_support()] # get_support() делает то, что даёт нужные столбцы
cols2 = cat_sc.columns[sel2.get_support()]
num_columns = num_columns[cols1]
categorical_columns = categorical_columns[cols2]

(1460, 27) (1460, 36)


In [38]:
from sklearn.feature_selection import mutual_info_regression # здесь выбираем лучшие признаки для классификации - chi2
need_cols = categorical_columns.columns[mutual_info_regression(categorical_columns,y_train)>0.05]
categorical_columns = categorical_columns[need_cols]
categorical_columns.shape # 13 категориальных - много, будем делать дерево или KNR

(1460, 13)

In [39]:
X_corr = num_columns.corr() # это - такой же DataFrame
hold = 0.8
to_drop = []
for col in X_corr.columns:
    cor_features = X_corr[col].abs()>hold # у Series вместо индексов - строки, но они ничего не делают. Берет абсолютные значения и смотрит у каждого, есть ли у него корреляция больше нужной
    cor_features[col] = False # не учитывается корреляция с самим собой
    if any(cor_features):  # Если есть хоть один признак, коррелирующий с текущим выше порога.
        to_drop.append(col)
num_columns = num_columns.drop(columns = to_drop)

In [40]:
from sklearn.feature_selection import mutual_info_regression # здесь выбираем лучшие признаки
need_cols = num_columns.columns[mutual_info_regression(num_columns,y_train)>0.05]
num_columns = num_columns[need_cols]
num_columns.shape

(1460, 20)

In [41]:
from sklearn.ensemble import RandomForestRegressor 
import pandas as pd
model = RandomForestRegressor(min_samples_leaf=4,random_state = 42) # *
X_train = pd.concat([num_columns,categorical_columns],axis=1)
#делаем модель для того, чтобы определить features_importance, она определяет, сколько неопределённости из данных уйдёт, если добавить этот признак (0.1 - сокращает на 10%)
model.fit(X_train,y_train)
threshold = 0.05  # Установим порог
selected_features = [i for i in range(X_train.shape[1]) if model.feature_importances_[i] > threshold] # *
X_train = X_train[X_train.columns[selected_features]]
X_train

,OverallQual,GrLivArea,Neighborhood
0,7.0,1710.0,197965.734807
1,6.0,1262.0,197643.209810
2,7.0,1786.0,197965.734807
3,7.0,1717.0,209344.287867
4,8.0,2198.0,318453.591177
...,...,...,...
1455,6.0,1647.0,192821.904993
1456,6.0,2073.0,189009.693995
1457,7.0,2340.0,209344.287867
1458,5.0,1078.0,145847.080044


In [42]:
X_test = X_test[X_train.columns]
X_test['Neighborhood'] = enc.fit_transform(X_test['Neighborhood'],y_test)
X_test[['OverallQual','GrLivArea']] = X_test[['OverallQual','GrLivArea']].fillna(X_test[['OverallQual','GrLivArea']].mean())
model.fit(X_train,y_train)
print(f'{model.score(X_test,y_test)} при {len(X_test.columns)} столбцов из 79'.center(70,'🎻'))

🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻0.8517243023809178 при 3 столбцов из 79🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻🎻
